In [1]:
import requests #used to read urls and extract information
from bs4 import BeautifulSoup as bts #webscraping library
import pandas as pd

In [2]:
URL = "https://www.bgr.in/gadgets/mobile-phones/search/"
res=[]  #save the final result 
no_of_pages=15 #no of pages that we are going to scrap

In [4]:
for _ in range(no_of_pages):
    r = requests.get(URL)                          #request library is used to  call url and get the content of html page
                                                   #using the beautifull soup to parse the html page next line
    soup = bts(r.content, 'html5lib')              #So basically we are fetching the html content that we get when "inspect element" is done. If this line causes an error, run 'pip install html5lib' or install html5lib
    a = soup.findAll("li",class_='mobile-listing') #When you click inspect element, you will see that all the informations are present in the "li mobile listing class". That's why here wen we do soup.findall, we will get all the codes that are present in that class. Get all the link of mobile specs 
                                                   #traverse over the mobile specific links to scrap the specs     
    for k in a:                                    #now for k in a that is soup.findall                                  
        data={}                                    #niw we will be creating a dictionary and saving all the specifications of each mobile here
                                                   
        url_tmp=k.find('a')['href']                #Here k is the loop variable. When k is 1, we are finding the information of the first phone in "a". "a" is the variable soup.findall that we did before, it has all the information and from that information we are extracting href. So href is the link that leads to all the detail of that k th mobile phone in "a". So we are basically getting this url here and storing it.
        mob=requests.get(url_tmp)                  #so again like how we did in the first for loop, we are fetching the url
        mob = bts(mob.content, 'html5lib')         #we are saving all the html info in mob with the help of bts library.
        
        info=mob.find_all('span',class_='col-xs-12 col-sm-5 spec-lbl')           #here we have the column names that is the informations are available in this class like SimType, Processor, Internal Memory and etc.
        desc=mob.find_all('span',class_='col-xs-12 col-sm-7 spec-val')           #here we have the details that should be filled in the columns like, DualSim, OctaCore, 4GB and etc. Match above and below (XimportantX)
        name=mob.find('h1',{'itemprop':'name'})                                  #the name of the mobile was not present in the above class, it was in a different class therefore we see that it's present in "h1" class so we are finding itemprop:name like how it was present in the html code
        data['name']=name.get_text()                                             #the name will be present like name: <h1 itemprop="name">Realme X3</h1> . Therefore we are using name.get_text() to get the name of the mobile phone and storing it in the dictinary with the key name as "name" and the value is the name of the mobile.
        price=mob.find("span",class_='rsm').get_text()                           #the price of the mobile was also not present in the col-xs... class therefore we are getting it here.
        
        for k,v in zip(info,desc):                                               #why are we zipping here? the info and desc are two lists that have values corresponding to one another, so we can zip it.
            data[k.get_text()]=v.get_text()        #YOU CAN UNDERSTAND THIS THINK!                              # k is info and v hold desc values.
        data['price']=price    
        res.append(data)                             # store the result into list "res". Look the list will be list of dictionaries like full of name:value, name:value continuously, where the name is gonna be the column name and value is gonna be the value of that column
    URL=soup.find('a',class_='next pagination')['href']            #get the url for moving to page 2


In [3]:
df = pd.DataFrame(res)
df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   name                     150 non-null    object
 1   Device Type              149 non-null    object
 2   Sim Capability           147 non-null    object
 3   Input Mechanism          150 non-null    object
 4   Availability             149 non-null    object
 5   Software                 149 non-null    object
 6   Processor                149 non-null    object
 7   Internal Memory          148 non-null    object
 8   Internal Storage         148 non-null    object
 9   Dedicated Micro SD Card  147 non-null    object
 10  3.5 mm Jack              141 non-null    object
 11  Display Technology       148 non-null    object
 12  Display Size             150 non-null    object
 13  Display Resolution       149 non-null    object
 14  Main Camera              147 non-null    o

In [ ]:
df.fillna('none',inplace=True)

In [ ]:
df

,name,Device Type,Sim Capability,Input Mechanism,Availability,Software,Processor,Internal Memory,Internal Storage,Dedicated Micro SD Card,...,Fast Charging,Wireless Charging,Network Type,Network Capability,Fingerprint Sensor,Face Unlock,price,IP Rating,SD Card Capacity,Dual Sim Capability
0,Realme 8i,Smartphone,Dual-SIM,Touchscreen,Available In India,realme UI 2.0 Based on Android 11,MediaTek Helio G96,4GB,64GB,Yes,...,Yes,No,GSM,"2G, 3G, 4G",Side-Mounted,Yes,"13,999",none,none,none
1,Realme 8s 5G,Smartphone,Dual-SIM,Touchscreen,Available In India,realme UI 2.0 Based on Android 11,MediaTek Dimensity 810 5G,6GB,128GB,Yes,...,Yes,No,GSM,"2G, 3G, 4G, 5G",Side-Mounted,Yes,"17,999",none,none,none
2,Xiaomi Redmi 10 Prime,Smartphone,Dual-SIM,Touchscreen,Available In India,MIUI 12.5 based on Android 11,MediaTek Helio G88,4GB,64GB,Yes,...,Yes,No,GSM,"2G, 3G, 4G",Side-Mounted,Yes,"12,499",none,none,none
3,Samsung Galaxy A52s 5G,Smartphone,Dual-SIM,Touchscreen,Available In India,Android 11,Qualcomm Snapdragon 778G,6GB,128GB,Yes,...,Yes,No,GSM,"2G, 3G, 4G",Under-Display,Yes,"35,999",none,none,none
4,Samsung Galaxy M32 5G,Smartphone,Dual-SIM,Touchscreen,Available In India,Android 11,MediaTek Dimensity 720,6GB,128GB,Yes,...,Yes,No,GSM,"2G, 3G, 4G, 5G",Side-Mounted,Yes,"20,999",none,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Vivo Y11,Smartphone,Dual-SIM,Touchscreen,Available In India,Funtouch OS 9 Based on Android 9,Qualcomm Snapdragon 439,3GB,32GB,Yes,...,No,No,GSM,"2G, 3G, 4G",Rear-Mounted,No,"9,990",none,256GB,none
146,Realme C11,Smartphone,Dual-SIM,Touchscreen,Available In India,Realme UI Based on Android 10,MediaTek Helio G35,2GB,32GB,Yes,...,No,No,GSM,"2G, 3G, 4G",No Fingerprint Sensor,No,"7,499",none,256 GB,none
147,Poco M2 Pro,Smartphone,Dual-SIM,Touchscreen,Available In India,MIUI 11 based on Android 10,Qualcomm Snapdragon 720G,4GB,64GB,Yes,...,Yes,No,GSM,"2G, 3G, 4G",Side-Mounted,Yes,"13,999",none,512GB,none
148,OnePlus Nord,Smartphone,Dual-SIM,Touchscreen,Available In India,OxygenOS based on Android 10,Qualcomm Snapdragon 765G,6GB,64GB,Yes,...,Yes,Select,GSM,"2G, 3G, 4G, 5G",Under-Display,Yes,"24,999",none,none,none


In [ ]:
df.shape

(150, 30)

In [ ]:
df['Display Resolution'].unique()

array(['1080x2412', '2400*1080', '2400 x 1080', '1080 x 2400', '720x1600',
       '1080x2408', '2400x1080', '720x1,560', '1080x2400', '1,080x2,640',
       '720x1,600', '2408×1080', '1080x2460', '1560 x 720', '1600X720',
       '1,080x2,460', '1,080x2,400', '2340 x 1080', '1080 x 2408',
       '1080*2460', '1600x720', '2340x1080 FHD+', '2400 x 1080 (FHD+)',
       '1600x720 (HD+)', '2404×1080 (FHD+)', '720 x 1600 (HD+)',
       '1440x3200', '2400x1080 FHD+', '1080×2400 FHD+', '1080×2400',
       '1080x2376', '3216 X 1440 pixels', '2400 x 1080 pixels',
       '1080x2400 pixels', '1,080x,2,400 pixels', '2448x1080 FHD+',
       '720x1,600 pixels', '1600 x 720 Pixels', '2400x1080 ',
       '2400 x 1080 FHD+', '1080x2340 pixels', '2400 x 1080FHD+',
       '720*1600', '1640 x 720', '1,080x2,400 pixels', '720x1,560 pixels',
       '720 x 1520 pixels', '2408 × 1080 (FHD+)', 'none', '1600*720',
       '2408 × 1080', '1560×720', '2400 x 1080 ', 'Full HD+',
       '720 x 1600 pixels', '1600×720',